In [ ]:
include("CheckOptionsNLEQ1.jl");

# Bookkeeping
# TODO: Make everything type independent
# Currently everything is assumed to be Float64
function nleq1(f,x,xscal,opt::OptionsNLEQ,wk::OptionsNLEQ)

    # TODO: persistent variables
    
    # Initialize output statistics
    stats = Dict{ASCIIString,Any}();

    # Initialize error code 0
    retCode = 0;

    # To print or not?
    printFlag = getOption!(opt,"PR_ERR",0);

    # Begin main
    
    # First call or successive call
    qsucc = getOption!(opt,"QSUCC",0);

    # Check input parameters and options
    n = length(x);
    retCode = checkOptions(n,x,xscal,opt);

    # Exit if any parameter error was detected
    if retCode != 0
        stats["Error_Code"] = retCode;
        return (stats, retCode)
    end

    # Check if this is a first call or successive call
    # to nleq1
    if qsucc != 0
        # TODO: check what this should be doing
    end

    # Check if the Jacobian is Dense/Sparse or Banded matrix
    mstor = getOption!(opt,"MSTOR",0);
    if mstor == 0
        m1 = n;
        m2 = n;
    elseif mstor == 1
        ml = getOption!(opt,"ML",0);
        mu = getOption!(opt,"MU",0);
        m1 = 2*ml + mu + 1;
        m2 = ml + mu + 1;
    end

    # User given Jacobian or not
    jacgen = getOption!(opt,"JACGEN",0);
    if jacgen == 0
        jacgen = 2;
    end
    opt.options["JACGEN"] = jacgen;

    qrank1 = getOption!(opt,"QRANK1",0);
    qordi  = getOption!(opt,"QORDI",0);
    qsimpl = getOption!(opt,"QSIMPL",0);

    if qrank1 == 1
        nbroy = getOption!(wk,"NBROY",0);
        if nbroy == 0
            nbroy = max(m2,10);
        end
        wk.options["NBROY"] = nbroy;
    else
        nbroy = 0;
    end

    # Workspace: WK

    initOption!(wk,"A",0.0);

    if qrank1 == 1
        wk.options["DXSAVE"] = zeros(n,nbroy);
    else
        wk.options["DXSAVE"] = 0.0;
    end

    initOptions!(wk,"DX" => zeros(n),"DXQ" => zeros(n),"XA" => zeros(n));
    initOptions!(wk,"XWA" => zeros(n),"F" => zeros(n),"FA" => zeros(n));
    initOptions!(wk,"ETA" => zeros(n),"XW" => zeros(n),"FW" => zeros(n));
    initOption!(wk,"DXQA",zeros(n));

    initOptions!(wk,"SUMXA0" => 0.0, "SUMXA1" => 0.0, "FCMON" => 0.0);
    initOptions!(wk,"FCMIN" => 0.0, "SIGMA" => 0.0, "SIGMA2" => 0.0);
    initOptions!(wk,"FCA" => 0.0, "FCKEEP" => 0.0, "FCPRI" => 0.0);
    initOptions!(wk,"DMYCOR" => 0.0, "CONV" => 0.0, "SUMX" => 0.0);
    initOptions!(wk,"DLEVF" => 0.0, "NITER" => 0, "NCORR" => 0);
    initOptions!(wk,"NFCN" => 0, "NJAC" => 0, "NFCNJ" => 0);
    initOptions!(wk,"NREJR1" => 0, "NEW" => 0, "ICONV" => 0);

    initOption!(opt,"NOROWSCAL", 0);

    # TODO: Print log of things done till now

    # Line 742 starts here
    nonlin = getOption!(opt,"NONLIN",3);
    initOption!(opt,"BOUNDEDDAMP",0);

    if opt.options["BOUNDEDDAMP"] == 0
        qbdamp = Int(nonlin == 4);
    elseif opt.options["BOUNDEDDAMP"] == 1
        qbdamp = 1;
    elseif opt.options["BOUNDEDDAMP"] == 2
        qbdamp = 0;
    end

    initOption!(wk,"FCBND",0.0);

    if qbdamp == 1
        if wk.options["FCBND"] < 1.0
            wk.options["FCBND"] = 10.0;
        end
    end

    # TODO: print somethings

    # Maximum permitted number of iteration steps
    nitmax = getOption!(wk,"NITMAX",50);
    if nitmax <= 0
        nitmax = 50;
    end
    wk.options["NITMAX"] = nitmax;

    # TODO: Print somethings

    # Initial damping factor for highly nonlinear problems
    initOption!(wk,"FCSTART",0.0);
    qfcstr = wk.options["FCSTART"] > 0.0;
    if !qfcstr
        wk.options["FCSTART"] = 1.0e-2;
        if nonlin == 4
            wk.options["FCSTART"] = 1.0e-4;
        end
    end

    # Minimal permitted damping factor
    initOption!(wk,"FCMIN",0.0);
    if wk.options["FCMIN"] <= 0.0
        wk.options["FCMIN"] = 1.0e-4;
        if nonlin == 4
            wk.options["FCMIN"] = 1.0e-8;
        end
    end
    fcmin = getOption!(wk,"FCMIN",0.0);

    # Rank1 decision parameter SIGMA
    initOption!(wk,"SIGMA",0.0);
    if wk.options["SIGMA"] < 1.0
        wk.options["SIGMA"] = 3.0;
    end
    if qrank1 == 0
        wk.options["SIGMA"] = 10.0/fcmin;
    end

    # Decision parameter about increasing too small predictor
    # to greater corrector value
    initOption!(wk,"SIGMA2",0.0);
    if wk.options["SIGMA2"] < 1.0
        wk.options["SIGMA2"] = 10.0/fcmin;
    end

    # Starting value of damping factor (fcmin <= fc <= 1.0)
    if nonlin <= 2 && !qfcstr
        # for linear or mildly nonlinear problems
        fc = 1.0;
    else
        # for highly or extremely nonlinear problems
        fc = getOption!(wk,"FCSTART");
    end

    # Simplified Newton iteration implies ordinary Newton iteration mode
    if getOption!(opt,"QSIMPL",0) == 1
        fc = 1.0;
    end

    # If ordinary Newton iteration, damping factor is always 1
    if getOption!(opt,"QORDI",0) == 1
        fc = 1.0;
    end

    wk.options["FCSTART"] = fc

    # Print something

    # Time measurement

    retCode = -1;

    # If retCode is unmodified on exit, successive steps are required
    # to complete the Newton iterations
    if nbroy == 0
        nbroy = 1;
    end

    # Call to n1int

    # Print statistics

    # set stats variable

    return (stats, retCode)
end

In [ ]:
include("CallTest.jl")

In [17]:
include("MachineConsts.jl")
function n1scal(n,x,xa,xscal,iscal,qinisc,opt::OptionsNLEQ)
    # Get machine constants required
    small =  getMachineConstants(6);
    
    if iscal == 1
        xw = xscal;
    else
        xw = max(xscal,max((abs(x)+abs(xa))*0.5,small*ones(n)));
    end
    
    # TODO: printing mumbo jumbo
    
    return xw
end

n1scal (generic function with 1 method)

In [50]:
# TODO: check the sparse part
function n1scrf(m,n,a)
    fw = zeros(n);
    if issparse(a)
        nza = nnz(a);
        (row,col,valA) = findnz(a);
        aout = sparse(row,col,zeros(nza),m,n);
        fw[row[1:nza]] = max(fw[row[1:nza]],abs(val));
        ## The above line replaces the for loop
        #for j = 1:nza
        #    rj = row[j];
        #    aaj = abs(a[rj,col[j]]);
        #    fw[rj] = max(fw[rj],aaj);
        #end
        ##
        for k = 1:m
            if fw[k] > 0.0
                fw[k] = 1.0/fw[k];
            else
                fw[k] = 1.0;
            end
        end
        
        aout[row,col] = valA.*fw[row[1:nza]];
        ## The above line replaces the for loop
        #for j = 1:nza
        #    aout[row[j],col[j]] = valA[j]*fw[row[j]];
        #end
    else
        aout = zero(a);
        for k = 1:m
            s1 = max(abs(a[k,1:n]));
            if s1 > 0.0
                s1 = 1.0/s1;
                fw[k] = s1;
                aout[k,1:n] = a[k,1:n]*s1;
            else
                fw[k] = 1.0;
                aout[k,1:n] = a[k,1:n];
            end
        end
    end
    
    return (aout,fw)
end

n1scrf (generic function with 1 method)

In [53]:
function n1scrb(n,lda,ml,mu,a)
    fw = zeros(n);
    aout = zero(a);
    m2 = ml + mu + 1;
    for k = 1:n
        s1 = 0.0;
        l2 = max(1, k - ml);
        l3 = min(n, k + mu);
        k1 = m2 + k;
        
        # TODO: consider replacing this loop
        for l1 = l2:l3
            s1 = max(s1,abs(a[k1-l1,l1]));
        end
        
        if s1 > 0.0
            s1 = 1.0/s1;
            fw[k] = s1;
            for l1 = l2:l3
                aout[k1-l1,l1] = a[k1-l1,l1]*s1;
            end
        else
            fw[k] = 1.0;
        end
    end
    
    return (aout,fw)
end

n1scrb (generic function with 1 method)

In [ ]:
function n1fact(n,lda,ml,mu,a,opt::OptionsNLEQ)
    # Does the LU factorization
    # TODO: Julia does not have a banaded mode for LU
    return (l,u,p,i)
end

In [ ]:
function n1jacb(fcn,par,n,lda,ml,x,fx,yscal,ajdel,ajmin,nfcn)
    # Evaluation of banded Jacobian matrix using finite
    # difference approximation adapted for use in nonlinear
    # systems solver NLEQ1
    return (a,nfcnnew,ifail)
end

function n1jcf(fcn,par,n,lda,x,fx,yscal,eta,etamin,etamax,etadif,conv,nfcn)
    # Approximation of dense Jacobian matrix for nonlinear systems solver
    # NLEQ1 with feed-back control of discretization and rounding errors
    return (a,eta,nfcnnew,ifail)
end

function n1jcfb(fcn,par,n,lda,ml,x,fx,yscal,eta,etamin,etamax,etadif,
                                           conv,nfcn)
    #Approximation of banded Jacobian matrix for nonlinear systems solver
    #NLEQ1 with feed-back control of discretization and rounding errors
    return (a,eta,nfcnnew,ifail)
end

In [ ]:
function n1prv1(dlevf,dlevx,fc,niter,new,mprmon,lumon,qmixio)
    # Printing of intermediate values type 1
    return nothing
end

function n1prv2(dlevf,dlevx,fc,niter,mprmon,lumon,qmixio,cmark)
    # Printing of intermediate values type 2
    return nothing
end

function n1sout(n,x,mode,opt::OptionsNLEQ,wk::OptionsNLEQ,mprint,luout)
    # Sol out printing of iterates
    return nothing
end

In [16]:
xw = max(xscal,max((abs(x)+abs(xa))*0.5,small*ones(n)))

4-element Array{Float64,1}:
 5.0
 5.0
 5.0
 5.0

In [40]:
x = sparse(eye(4));
(row,col,val) = findnz(x);

In [41]:
nza = nnz(x);

In [42]:
m = 4; n = 4;

In [43]:
aout = sparse(row,col,zeros(nza),m,n);

In [46]:
fw = zeros(6);

In [49]:
fw[row[1:nza]] = max(fw[row[1:nza]],abs(val));

In [39]:
find(fw)

4x4 sparse matrix with 4 Float64 entries:
	[1, 1]  =  1.0
	[2, 2]  =  1.0
	[3, 3]  =  1.0
	[4, 4]  =  1.0

In [52]:
max(2,3)

3